# How this filter works
- 1. Assign `all_plugins.json` and to variable `all_plugins` to it
- 2. Create `blacklist_list` dict
  - 1. Get blacklist from `blacklist.json`
  - 2. Since `blacklist.json` is a nested dict, only get the leaf arrays and aggregate them into a single array. Set this array to `blacklist_list`
- 3. Assign `plugins_info.json` to variable `plugins_info`
- 4. Initiate a dict called inital_screening. Execute an initial screening of plugins with the following loop:
  - 1. Create a dict called `plugin_info` with the item `{namespace: plugin["namespace"], image: plugin["manifest"].get("logo_url", None), description_for_human: plugin["manifest"]["description_for_human"], description_for_model[plugin["manifest"]["description_for_model"]], openapi_url: plugin["manifest"]["api"]["url"]}`
  - 2. Check if `auth` is `"none"`
    - 1. If passes then add the following items to `plugin_info` to `{auth: false}`
    - 2. If fails then add the following items to `plugin_info` to `{auth: true}`
  - 3. Check that `namespace` is not it `blacklist_list`
    - 1. If passes then add the following items to `plugin_info` to `{blacklisted: false}`
    - 2. If fails then add the following items to `plugin_info` to `{blacklisted: true}`
  - 4. Add `plugin_info` to `plugins_info` with key of the namespace
- 5. Iterate through `plugins_info` and do the following:
  - 1. Check if `plugin_info["openapi_url"]` can be called without error
    - 1. If passes then add the following items to `plugin_info` to `{whitelisted: true}`
    - 2. If fails then add the following items to `plugin_info` to `{whitelisted: false}`
- 6. iterate through `openplugin_classes` to do a complete prompted test.
  - 1. generate a stimulous prompt to test the plugin
  - 2. use stimulous prompt to see if plugin can be called without error
- 7. Creaate a status for all plugins
  - 1. all plugins start with status `unsupported`
  - 2. if plugin `not openplugin_info["auth"] and not openplugin_info["blacklisted"] and openplugin_info["whitelisted"]` then status is `tentative`
  - 3. if plugin has `openplugin_info["stimulous_prompt"] and openplugin_info["stimulated"]` the status is `supported`
- 8. Create a file called `openplugins_compressed.json` and save only plugins that are `tentative` or `supported`
  - 1. Create a dict called `openplugins_compressed`
  - 2. Iterate through `plugins_info` and do the following:
    - 1. Check if `plugin_info["status"]` is `tentative` or `supported`
      - 1. If passes then add item {[namespace]: "https://" + openplugin_info["domain"]} to `openplugins_compressed`
- 9. create `PLUGINS.md` table based on `openplugins_compressed`

In [2]:
import json
import os
import re
import requests
from openplugincore import OpenPlugin
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# 1. Assign `all_plugins.json` to variable `all_plugins`
pypi_client_path = 'all_plugins.json'

with open(pypi_client_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Concatenate lines into a single JSON document
json_content = ''.join(lines)

try:
    all_plugins = json.loads(json_content)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")

print(f"Number of plugins: {len(all_plugins)}\nFirst plugin: {json.dumps(all_plugins[0], indent=2)}")

Number of plugins: 615
First plugin: {
  "id": "plugin-985f0d2e-45f1-495c-b293-72ec96b1857f",
  "domain": "diary-chatgptplugin.derekxwang.com",
  "namespace": "diary",
  "status": "approved",
  "manifest": {
    "schema_version": "v1",
    "name_for_model": "diary",
    "name_for_human": "Diary",
    "description_for_model": "This plugin offers prompts for seamless diary management, coupled with a wealth of useful information related to maintaining a diary. It is particularly useful when users are working on their diaries and journals.",
    "description_for_human": "Kickstart your diary journey. Keeping a diary is designed to be effortless and beneficial.",
    "auth": {
      "type": "service_http",
      "instructions": "",
      "authorization_type": "bearer",
      "verification_tokens": {
        "openai": "6f4a6ab634264bda98df697b672b90c9"
      }
    },
    "api": {
      "type": "openapi",
      "url": "https://diary-chatgptplugin.derekxwang.com/.well-known/openapi.yaml"
    }

In [5]:
# 2. Create `blacklist_list` dict
with open('blacklist.json', 'r', encoding='utf-8') as f:
    blacklist = json.load(f)
def aggregate_items(json_object):
    aggregate_list = []
    for key in json_object:
        aggregate_list.extend(json_object[key])
    return aggregate_list
blacklist_list = aggregate_items(blacklist)
print(f"Number of blacklisted plugins: {len(blacklist_list)}\nBlacklist list: {blacklist_list}")

Number of blacklisted plugins: 23
Blacklist list: ['Coursera', 'video_insights', 'KAYAK', 'ArtCollection', 'AskMarcie', 'Avalara', 'BOXIL_SaaS', 'Broadway', 'BubbleGoods', 'ChatWithBible', 'ChatWithQuran', 'CryptoPrices', 'FreshTech', 'ABCmouse', 'staypia', 'DAIZY', 'Company_Transcripts', 'Bohita', 'CheckTheChain', 'Checkers', 'CloudDiagramGen', 'Diagrams', 'Creaticode_Extension_of_MIT_Scratch']


In [6]:
# 3. Assign `plugins_info.json` to variable `plugins_info`
with open('openplugins_info.json', 'r', encoding='utf-8') as f:
    openplugins_info: dict = json.load(f)
print(f"Number of plugins in openplugins_info: {len(openplugins_info)}")

Number of plugins in openplugins_info: 609


In [45]:
# 4. Initiate a dict called inital_screening. Execute an initial screening of plugins with the following loop:
for plugin in all_plugins:
    openplugin_info = {
        **openplugins_info.get(plugin['namespace'], {}),
        'namespace': plugin['namespace'],
        'image': plugin['manifest'].get('logo_url', None),
        'description_for_human': plugin['manifest'].get('description_for_human', None),
        'description_for_model': plugin['manifest'].get('description_for_model', None),
        'domain': plugin['domain'],
        'openapi_url': plugin['manifest'].get('api', {}).get('url', None)
    }
    if plugin['manifest'].get('auth', {}).get('type', None) == 'none':
        openplugin_info['auth'] = False
    else:
        openplugin_info['auth'] = True
    if plugin['namespace'] not in blacklist_list:
        openplugin_info['blacklisted'] = False
    else:
        openplugin_info['blacklisted'] = True
    openplugins_info[plugin['namespace']] = openplugin_info
with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"Number of plugins in openplugins_info: {len(openplugins_info)}\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")


Number of plugins in openplugins_info: 610
First openplugin info: {
  "auth": false,
  "blacklisted": true,
  "description_for_human": "Provides fun and educational learning activities for children 2-8 years old.",
  "description_for_model": "Assistant uses the ABCmouse plugin to get relevant learning activity suggestions for any child 2-8 years of age. Assistant will reply with the following 3 paragraphs 1) Activity Search Results [image] and [url] 2) Activity Learning Objectives [description] and [learning_objective] 3) Followup Questions. The first paragraph contains a list of the activities [url] with their learning attributes listed clearly and concisely as bullet points under the product [description], together with a link to the activity [url] and an explanation [learning_objective]. Links will always be returned and should be shown to the user. Assistant suggestions consider only the most important objectives of the activities [description, learning_objective] that will help th

In [62]:
# 5. Iterate through `plugins_info` and do the following:
openplugin_classes = {}
for idx, (namespace, openplugin_info) in enumerate(openplugins_info.items()):
    if idx % 20 == 0:
        print(f"Processing plugin {idx} of {len(openplugins_info)}")
    # (namespace, plugin_info) = list(openplugins_info.items())[12]
    root_url = "https://" + openplugin_info['domain']
    try:
        openplugin_classes[namespace] = OpenPlugin("", root_url=root_url)
        openplugin_info['whitelisted'] = True
    except Exception as e:
        print(f"Error: {e}")
        openplugin_info['whitelisted'] = False
    openplugins_info[namespace] = openplugin_info
with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]

whitelisted_count = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info['whitelisted'] == True:
        whitelisted_count += 1
print(f"{whitelisted_count} whitelisted plugins of out of {len(openplugins_info)}\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")


Processing plugin 0 of 609


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better s

Processing plugin 20 of 609


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better s

Processing plugin 40 of 609


...

Processing plugin 580 of 609


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better s

Processing plugin 600 of 609


Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better s

609 whitelisted plugins of out of 609
First openplugin info: {
  "auth": false,
  "blacklisted": true,
  "description_for_human": "Provides fun and educational learning activities for children 2-8 years old.",
  "description_for_model": "Assistant uses the ABCmouse plugin to get relevant learning activity suggestions for any child 2-8 years of age. Assistant will reply with the following 3 paragraphs 1) Activity Search Results [image] and [url] 2) Activity Learning Objectives [description] and [learning_objective] 3) Followup Questions. The first paragraph contains a list of the activities [url] with their learning attributes listed clearly and concisely as bullet points under the product [description], together with a link to the activity [url] and an explanation [learning_objective]. Links will always be returned and should be shown to the user. Assistant suggestions consider only the most important objectives of the activities [description, learning_objective] that will help them fi

In [63]:
# 6. iterate through `openplugin_classes`
for idx, (namespace, openplugin_class) in enumerate(openplugin_classes.items()):
    # (namespace, openplugin_class) = list(openplugin_classes.items())[0]
    if idx % 20 == 0:
        print(f"Processing plugin {idx} of {len(openplugins_info)}")
    openplugin_info = openplugins_info[namespace]
    openplugin_info["stimulous_prompt"] = None
    openplugin_info["stimulated"] = False
    if not openplugin_info["auth"] and not openplugin_info["blacklisted"] and openplugin_info["whitelisted"]:
        try:
            if openplugin_info.get("stimulous_prompt", None) is None:
                generate_stimulation_prompt_prompt = {
                    "prompt": f"""
                    Please create a prompt that will trigger an model's plugin with the human description delimited by driple backticks.
                    If necessary also look at the model description also delimited by triple backticks.
                    Please do not ask anything from the AI you should provide all the information it needs in the prompt.
                    You should not be ambiguous or open ended in your prompt use specific examples.
                    Do not simply restate the description.
                    Human description:
                    ```
                    {openplugin_info["description_for_human"]}
                    ```
                    Model description:
                    ```
                    {openplugin_info["description_for_model"]}
                    ```
                    """,
                    "function": {
                    "name": "stimulous_prompt_generation",
                    "description": """
                    Generates a natural language phrase to that triggeres the AI plugin.
                    If approriate the phrase should include an example item/url (https://github.com/)/text/ect. even if you are not sure if it is real its ok to make it up.
                    """,
                    "parameters": {
                        "type": "object",
                        "properties": {
                        "stimulous_prompt": {
                            "type": "string",
                            "description": "The stimulous phrase to trigger the AI plugin"
                        },
                        },
                        "required": ["stimulous_prompt"]
                    }
                    }
                }
                generation = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo-0613",
                    temperature=0,
                    messages=[{"role": "user", "content": generate_stimulation_prompt_prompt["prompt"]}],
                    functions=[generate_stimulation_prompt_prompt["function"]],
                    function_call={"name": "stimulous_prompt_generation"}
                )
                json_arguments = json.loads(generation["choices"][0]["message"]["function_call"]["arguments"])
                openplugin_info["stimulous_prompt"] = json_arguments["stimulous_prompt"]
            if  openplugin_info["stimulous_prompt"]:
                function_response = openplugin_class.fetch_plugin(
                    prompt=openplugin_info["stimulous_prompt"],
                    model="gpt-3.5-turbo-0613",
                    temperature=0,
                )
                openplugin_info["stimulated"] = True
        except Exception as e:
            print(f"Plugin '{namespace}' failed with error:\n{e}")
with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)

num_stimulous_prompt = 0
num_stimulated = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["stimulous_prompt"]:
        num_stimulous_prompt += 1
    if openplugin_info["stimulated"]:
        num_stimulated += 1
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"{num_stimulous_prompt} plugins with stimulous prompts, {num_stimulated} plugins stimulated, out of {len(openplugins_info)} plugins\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")
    







Processing plugin 0 of 609
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'psychologist'}}}
Plugin 'AIMaster' failed with error:
Invalid URL '//query/prompt/psychologist': No scheme supplied. Perhaps you meant https:////query/prompt/psychologist?
Plugin 'AINewsRoundup' failed with error:
Expecting value: line 1 column 1 (char 0)
Plugin 'AMAZON_product_desc_generator' failed with error:
Not a plugin function
Plugin 'Agones' failed with error:
Not a plugin function
Plugin 'Ai_PDF' failed with error:
Not a plugin function
Plugin 'Algorithma' failed with error:
Not a plugin function
llm_chain_out:  {'name': 'get_openapi_yaml_openapi_yaml_get', 'arguments': {}}
Plugin 'ApexMap' failed with error:
Invalid URL '//openapi.yaml': No scheme supplied. Perhaps you meant https:////openapi.yaml?
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'AI-powered Text-to-App Generator turn

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Plugin 'questmate' failed with error:
Not a plugin function
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'pasta dish'}}}
Plugin 'recipe_retrieval' failed with error:
Invalid URL '//query/prompt/pasta dish': No scheme supplied. Perhaps you meant https:////query/prompt/pasta dish?
Plugin 'rentable_apartments' failed with error:
Not a plugin function
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'perfect'}}}
Plugin 'rephrase' failed with error:
Invalid URL '//query/prompt/perfect': No scheme supplied. Perhaps you meant https:////query/prompt/perfect?
Plugin 'repo_inspector' failed with error:
Not a plugin function
Plugin 'repo_radar' failed with error:
Not a plugin function
Plugin 'resume' failed with error:
Not a plugin function
Plugin 'reviewreader' failed with error:
Not a plugin function
Processing plugin 500 of 609


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'latest stock price for Apple Inc.'}}}
Plugin 'savvy_trader_ai' failed with error:
Invalid URL '//query/prompt/latest stock price for Apple Inc.': No scheme supplied. Perhaps you meant https:////query/prompt/latest stock price for Apple Inc.?
Plugin 'scholarai' failed with error:
Not a plugin function
llm_chain_out:  {'name': 'scholarly.search_paper', 'arguments': {'query': 'summary of research paper'}}
Plugin 'scholarly' failed with error:
Not a plugin function
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'COVID-19'}}}
Plugin 'science' failed with error:
Invalid URL '//query/prompt/COVID-19': No scheme supplied. Perhaps you meant https:////query/prompt/COVID-19?
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'design course'}}}
Plugin 'search' failed with er

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 05 Jul 2023 00:18:18 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7e1b667daff21140-ORD', 'alt-svc': 'h3=":443"; ma=86400'}.


Plugin 'surge_ai_trends' failed with error:
Not a plugin function
Plugin 'tailor_erp' failed with error:
Not a plugin function
Processing plugin 540 of 609
Plugin 'talkfpl' failed with error:
Not a plugin function
Plugin 'talkwithdocs' failed with error:
Not a plugin function
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'waiting time Disneyland California'}}}
Plugin 'themeparkhipster' failed with error:
Invalid URL '//query/prompt/waiting time Disneyland California': No scheme supplied. Perhaps you meant https:////query/prompt/waiting time Disneyland California?
Plugin 'timemachine' failed with error:
Not a plugin function
llm_chain_out:  {'name': 'get_prompts_query_prompt__keyword__get', 'arguments': {'path_params': {'keyword': 'beauty products'}}}
Plugin 'tira' failed with error:
Invalid URL '//query/prompt/beauty products': No scheme supplied. Perhaps you meant https:////query/prompt/beauty products?
llm_chain_out:  {'na

In [64]:
# 7. Creaate a status for all plugins
for namespace, openplugin_info in openplugins_info.items():
    openplugin_info["status"] = "unsupported"
    if not openplugin_info["auth"] and not openplugin_info["blacklisted"] and openplugin_info["whitelisted"]:
        openplugin_info["status"] = "tentative"
    if openplugin_info["stimulous_prompt"] and openplugin_info["stimulated"]:
        openplugin_info["status"] = "supported"
    openplugins_info[namespace] = openplugin_info
with open('openplugins_info.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_info, f, indent=2, sort_keys=True)

num_unsupported = 0
num_tentative = 0
num_supported = 0
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["status"] == "unsupported":
        num_unsupported += 1
    if openplugin_info["status"] == "tentative":
        num_tentative += 1
    if openplugin_info["status"] == "supported":
        num_supported += 1
first_openplugin_info = openplugins_info[list(openplugins_info.keys())[0]]
print(f"{num_unsupported} plugins with status unsupported, {num_tentative} plugins with status tentative, {num_supported} plugins with status supported, out of {len(openplugins_info)} plugins\nFirst openplugin info: {json.dumps(first_openplugin_info, indent=2)}")

184 plugins with status unsupported, 425 plugins with status tentative, 0 plugins with status supported, out of 609 plugins
First openplugin info: {
  "auth": false,
  "blacklisted": true,
  "description_for_human": "Provides fun and educational learning activities for children 2-8 years old.",
  "description_for_model": "Assistant uses the ABCmouse plugin to get relevant learning activity suggestions for any child 2-8 years of age. Assistant will reply with the following 3 paragraphs 1) Activity Search Results [image] and [url] 2) Activity Learning Objectives [description] and [learning_objective] 3) Followup Questions. The first paragraph contains a list of the activities [url] with their learning attributes listed clearly and concisely as bullet points under the product [description], together with a link to the activity [url] and an explanation [learning_objective]. Links will always be returned and should be shown to the user. Assistant suggestions consider only the most important

In [7]:
# 8. Create a file called `openplugins_compressed.json` and save only plugins that are `tentative` or `supported`
openplugins_compressed = {}
for namespace, openplugin_info in openplugins_info.items():
    if openplugin_info["status"] == "tentative" or openplugin_info["status"] == "supported":
        openplugins_compressed[namespace] = "https://" + openplugin_info["domain"]

with open('openplugins_compressed.json', 'w', encoding='utf-8') as f:
    json.dump(openplugins_compressed, f, indent=2, sort_keys=True)

first_openplugin_compressed = openplugins_compressed[list(openplugins_compressed.keys())[0]]
print(f"{len(openplugins_compressed)} plugins in openplugins_compressed\nFirst openplugin compressed: {json.dumps(first_openplugin_compressed, indent=2)}")

425 plugins in openplugins_compressed
First openplugin compressed: "https://plugin.chat2any.com"


In [9]:
# 9. create PLUGINS.md table
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir))

# Create the file path for plugins.md
pluginsmd_path = os.path.join(root_dir, 'PLUGINS.md')
plugins_md = """
# Plugins
Available plugins for OpenPlugin
Status:
- `tentative`: passed basic tests (may work)
- `supported`: passed complete prompt tests (should to work)

| Image | Namespace | Status | Description | Description for model |
| --- | --- | --- | --- | --- |
"""

def escape_special_markdown_chars(text):
    # Characters to escape: \ ` * _ { } [ ] ( ) # + !
    special_chars = r'\\|`|\*|_|{|}|\[|\]|\(|\)|#|\+|!'
    return re.sub(special_chars, lambda match: '\\' + match.group(), text)

def remove_line_breaks(text):
    return text.replace('\n', ' ').replace('\r', '')

for namespace, _root_url  in openplugins_compressed.items():
        openplugin_info = openplugins_info[namespace]
        if openplugin_info["image"]:
            image = escape_special_markdown_chars(openplugin_info["image"])
        else:
            image = escape_special_markdown_chars("https://i.imgur.com/L3giCRt.png")
        namespace = escape_special_markdown_chars(openplugin_info["namespace"])
        status = escape_special_markdown_chars(openplugin_info["status"])
        description = escape_special_markdown_chars(remove_line_breaks(openplugin_info["description_for_human"]))
        description_for_model = escape_special_markdown_chars(openplugin_info["description_for_model"])
        plugins_md += f"| ![{namespace} Logo]({image}) | {namespace} | {status} | {description} | {description_for_model} |\n"

with open(pluginsmd_path, 'w', encoding='utf-8') as f:
    f.write(plugins_md)

print(f"Successfully created PLUGINS.md file with {len(openplugins_compressed)} plugins of {len(openplugins_info)} total plugins.")

Successfully created PLUGINS.md file with 425 plugins of 609 total plugins.
